<a href="https://colab.research.google.com/github/mapmatters/coursera-nlp/blob/main/nlp_class_1_Week1_nb04_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/coursera/nlp')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'nlp class 1 - Week1 - assignment'  'nlp class 1 - Week1 - nb02'   py
'nlp class 1 - Week1 - nb01'	    'nlp class 1 - Week1 - nb03'   __pycache__


In [ ]:
import nltk                         # NLP toolbox
from os import getcwd
import pandas as pd                 # Library for Dataframes 
from nltk.corpus import twitter_samples 
import matplotlib.pyplot as plt     # Library for visualization
import numpy as np                  # Library for math functions

In [ ]:
# import my functions
import sys
sys.path.append('py')
import my_utils as my
from my_utils import process_tweet, build_freqs, extract_features

In [ ]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [ ]:
filepath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filepath)

In [ ]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [ ]:
# split the data into two pieces
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [ ]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos),1)), np.zeros((len(train_neg),1)), axis=0)
test_y = np.append(np.ones((len(test_pos),1)), np.zeros((len(test_neg),1)), axis=0)

In [ ]:
print("train_y.shape = ", train_y.shape)
print("test_y.shape = ", test_y.shape)

train_y.shape =  (8000, 1)
test_y.shape =  (2000, 1)


In [ ]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

print("type(freqs) = ", str(type(freqs)))
print("len(freqs) = ", str(len(freqs.keys())))

type(freqs) =  <class 'dict'>
len(freqs) =  11346


In [ ]:
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

In [ ]:
# example 
print(train_x[0])
print(process_tweet(train_x[0]))

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


### Part 1 - Logistic Regression

In [ ]:
def sigmoid(z):
  h = 1 / (1 + np.exp(-z))
  return h

### Part 1.2 Cost function and Gradient

In [ ]:
def loss(h,y,m):
  ls = (-1)*( np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)) )
  ls = ls/m
  return ls

In [ ]:
print("when actual=0, predict=0.9999, loss="+ str((-1)*(1-0)*np.log(1-0.9999)))
print("when actual=1, predict=0.0001, loss="+ str((-1)*(1)*np.log(0.0001)))

when actual=0, predict=0.9999, loss=9.210340371976294
when actual=1, predict=0.0001, loss=9.210340371976182


In [ ]:
# gradient descent
def gradientDescent(x, y, theta, alpha, num_iters):
  """
  input
    x: matrix of features
    y: labels
    theta: weight vector
    alpha: learning rate
    num_iters: number of iteration
  output
    J = final cost
    theta: final vector
  """
  m = len(x)
    
  for i in range(0, num_iters):
      
      # get z, the dot product of x and theta
      z = np.dot(x, theta)

      # get the sigmoid of z
      h = 1 / (1 + np.exp(-z))

      # calculate the cost function
      J = (-1/m)*( np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)) )

      # update the weights theta
      theta = theta - (alpha/m)*(np.dot(x.T, (h.reshape(m,1)-y)))
      
  ### END CODE HERE ###
  J = float(J)
  return J, theta

In [ ]:
def extract_features_new(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

### Part 3. Training your model

In [ ]:
# prepare data
X = np.zeros((len(train_x),3))
for i in range(len(train_x)):
  X[i,:] = extract_features(process_tweet(train_x[i]), freqs)
Y = train_y
X

array([[1.000e+00, 3.020e+03, 6.100e+01],
       [1.000e+00, 3.573e+03, 4.440e+02],
       [1.000e+00, 3.005e+03, 1.150e+02],
       ...,
       [1.000e+00, 1.440e+02, 7.830e+02],
       [1.000e+00, 2.050e+02, 3.890e+03],
       [1.000e+00, 1.890e+02, 3.974e+03]])

In [ ]:
theta = np.zeros((3,1))
print(f"initial theta is {theta.T}")
J, theta = gradientDescent(X, Y, theta, 1e-9, 1500)
print(f"the cost after training is {J:.8f}")
print(f"the resulting vector fo weights is {[round(t,8) for t in np.squeeze(theta)]}")

initial theta is [[0. 0. 0.]]
the cost after training is 0.24216529
the resulting vector fo weights is [7e-08, 0.0005239, -0.00055517]


### Test logistic regression

In [ ]:
def predict_tweet(tweet, freqs, theta):
  """
  input
    tweet : a string
    freqs : dictionary
    theta : vector weight
  output
    y_pred
  """
  x = extract_features_new(tweet, freqs)

  y_pred = 1 / (1 + np.exp(-np.dot(x, theta)))
  return y_pred

### Performance using test set

In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
  """
  input
    test_x
    test_y
    freqs
    theta
  output
    accuracy
  """
  y_hat = []

  for tweet in test_x:
    y_pred = predict_tweet(tweet, freqs, theta)
    if y_pred > 0.5:
      y_hat.append(1.0)
    else:
      y_hat.append(0)
  y_hat = np.asarray(y_hat).reshape(len(test_y),1)
  # result = y_hat.T==test_y
  accuracy = np.sum(y_hat==test_y) / len(test_y)
    
  return accuracy, y_hat

In [ ]:
sample = predict_tweet(test_x[0], freqs, theta)
print(sample)

[[0.81060847]]


In [ ]:
tmp_accuracy, y_pred_arrays = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


In [ ]:
y_pred_arrays[10]

array([1.])

### error analysis

In [ ]:
print("label predicted tweet")
for x, y in zip(test_x, test_y):
  y_hat = predict_tweet(x, freqs, theta)

  if  np.abs(y- (y_hat>0.5)) > 0:
    print("\nTHE TWEET IS:", x)
    print("THE PROCESS TWEET IS: ", x)
    print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

label predicted tweet

THE TWEET IS: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
THE PROCESS TWEET IS:  @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
1	0.49996890	b'truli later move know queen bee upward bound movingonup'

THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESS TWEET IS:  @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
1	0.48622857	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'

THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESS TWEET IS:  I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
1	0.483

In [ ]:
import inspect
print(inspect.getsource(extract_features))